In [2]:
# !pip install torch==1.10
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.html
# !pip install torch-geometric
# !pip install ipykernel
# !pip install wandb

In [3]:
import wandb

wandb.init(project="relational-autoencoder", entity="link-prediction-gnn", )
wandb.config = {
  "learning_rate": 0.005,
  "epochs": 25,
}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: di3go (use `wandb login --relogin` to force relogin)
/Users/di3go/Desktop/neural-graph/venv/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [4]:
import torch, torch_geometric

In [5]:
DS_PATH= './FB15k-237'
DS_NAME= 'FB15k-237'
ds = torch_geometric.datasets.RelLinkPredDataset(DS_PATH, DS_NAME)

edge_indices_train = ds.data.edge_stores[0]['train_edge_index']
edge_types_train = ds.data.edge_stores[0]['train_edge_type']

edges_test = ds.data.edge_stores[0]['test_edge_index'].detach()
edges_type_test = ds.data.edge_stores[0]['test_edge_type'].detach()

In [6]:
N_RELATIONS = 237
N_NODES = ds.data.num_nodes
NODE_EMBEDDING_SIZE = 16

rgcn = torch_geometric.nn.RGCNConv(N_NODES, NODE_EMBEDDING_SIZE, N_RELATIONS)
r_matrices = torch.randn(N_RELATIONS, NODE_EMBEDDING_SIZE, NODE_EMBEDDING_SIZE)


In [7]:
import matplotlib.pyplot as plt

# Create binary dense matrix for relation type == 1

train_links_truth = torch.sparse_coo_tensor(edge_indices_train, edge_types_train, (N_NODES,N_NODES) ).to_dense().float()
# test_links_truth = torch.sparse_coo_tensor(Xi_test[:,test_f], Xr_test[test_f], (N_NODES,N_NODES)).to_dense().float()

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(rgcn.parameters(), wandb.config['learning_rate'])

for epoch in range(wandb.config['epochs']):
    nodes_embeddings = rgcn(None, edge_indices_train, edge_types_train)
    optimizer.zero_grad()
    # edges_scores = torch.zeros((N_RELATIONS, N_NODES, N_NODES))
    tot_loss = torch.zeros(1)
    tot_acc = 0
    for r_id, r_mat in enumerate(r_matrices):
        # print(f'relation {r_id}')
        edge_scores = nodes_embeddings @ r_mat @ nodes_embeddings.T
        # print('scored', edge_scores)
        edge_tags = edge_scores.sigmoid().round()
        # print('tagged', edge_tags)

        r_indices = edge_indices_train[:, edge_types_train == r_id]
        edge_tags_truth = torch.zeros((N_NODES, N_NODES))
        edge_tags_truth[r_indices] = 1
        
        loss = criterion(edge_scores, edge_tags_truth)
        acc = ( 1 - (edge_tags - edge_tags_truth).abs()).sum() / (N_NODES**2)
        
        tot_loss += loss
        tot_acc += (tot_acc * (r_id - 1) + acc ) / r_id
        print(f"Epoch: {epoch} | RID: {r_id} | Acc: {acc} ")

        if r_id > 1:
            break

    tot_loss.backward()
    optimizer.step()
    wandb.log({"loss": loss})
    wandb.log({"accuracy": acc})


Epoch: 0 | RID: 0 | Acc: 0.500702977180481 
Epoch: 0 | RID: 1 | Acc: 0.5009181499481201 
Epoch: 0 | RID: 2 | Acc: 0.4986039102077484 
Epoch: 1 | RID: 0 | Acc: 0.38951170444488525 
Epoch: 1 | RID: 1 | Acc: 0.409082293510437 
Epoch: 1 | RID: 2 | Acc: 0.2604365646839142 
Epoch: 2 | RID: 0 | Acc: 0.7338242530822754 
Epoch: 2 | RID: 1 | Acc: 0.5953525304794312 
Epoch: 2 | RID: 2 | Acc: 0.7759602665901184 
Epoch: 3 | RID: 0 | Acc: 0.9019911885261536 
Epoch: 3 | RID: 1 | Acc: 0.6864050030708313 
Epoch: 3 | RID: 2 | Acc: 0.8584794402122498 
Epoch: 4 | RID: 0 | Acc: 0.9749388098716736 
Epoch: 4 | RID: 1 | Acc: 0.7927309274673462 
Epoch: 4 | RID: 2 | Acc: 0.8829113841056824 
Epoch: 5 | RID: 0 | Acc: 0.985205352306366 
Epoch: 5 | RID: 1 | Acc: 0.8687930703163147 
Epoch: 5 | RID: 2 | Acc: 0.9222114682197571 
Epoch: 6 | RID: 0 | Acc: 0.9881077408790588 
Epoch: 6 | RID: 1 | Acc: 0.9233533143997192 
Epoch: 6 | RID: 2 | Acc: 0.9508810043334961 
Epoch: 7 | RID: 0 | Acc: 0.9891273379325867 
Epoch: 7 | R

wandb: Network error (ConnectionError), entering retry loop.
